<a href="https://colab.research.google.com/github/techwithanirudh/Shared/blob/master/DeepFake_Website_TooHappyPart2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask_ngrok flask_cors

In [5]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model
!cd first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256
Receiving objects: 100% (256/256), 72.13 MiB | 23.02 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os

# os.chdir('/content/gdrive/MyDrive')
# !unzip first-order-motion-model.zip
os.chdir('/content/first-order-model')

In [3]:
os.mkdir('/content/templates')
os.chdir('/content/templates')

In [9]:
with open('index.html', 'w') as f:
  f.write('''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
    <title>Document</title>
    <style>
        body{
          background: rgba(0,0,0,0.9);
        }
        form{
          position: absolute;
          top: 50%;
          left: 50%;
          margin-top: -100px;
          margin-left: -250px;
          width: 500px;
          height: 200px;
          border: 4px dashed #fff;
        }
        form p{
          width: 100%;
          height: 100%;
          text-align: center;
          line-height: 170px;
          color: #ffffff;
          font-family: Arial;
        }
        form input{
          position: absolute;
          margin: 0;
          padding: 0;
          width: 100%;
          height: 100%;
          outline: none;
          opacity: 0;
        }
        form button{
          margin: 0;
          color: #fff;
          background: #16a085;
          border: none;
          width: 508px;
          height: 35px;
          margin-top: -20px;
          margin-left: -4px;
          border-radius: 4px;
          border-bottom: 4px solid #117A60;
          transition: all .2s ease;
          outline: none;
        }
        form button:hover{
          background: #149174;
        	color: #0C5645;
        }
        form button:active{
          border:0;
        }
    </style>
    <noscript>
        <style type="text/css">
            .pagecontainer {display:none;}
        </style>
        <div class="noscriptmsg">
        You don't have javascript enabled!
        Turn on javascript to continue to this page.
        </div>
    </noscript>
</head>
<body>
    <form action="/upload" method="POST" enctype="multipart/form-data">
        <input type="file" name="file" multiple>
        <p>Drag your files here or click in this area.</p>
        <button type="submit">Upload</button>
    </form>

    <script>
        $(document).ready(function(){
          $('form input').change(function () {
            $('form p').text(this.files.length + " file(s) selected");
          });
        });
    </script>
</body>
</html>
  ''')
f.close()

In [12]:
with open('progress.html', 'w') as f:
  f.write('''

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
    <style>
        body{
          background: rgba(0,0,0,0.9);
        }
        .progress-line, .progress-line:before {
          height: 20px;
          width: 100%;
          margin: 0;    
        }
        .progress-line {
          background-color: #b3d4fc;
          display: -webkit-flex;
          display: flex;
        }
        .progress-line:before {
          background-color: #3f51b5;
          content: '';
          -webkit-animation: running-progress 2s cubic-bezier(0.4, 0, 0.2, 1) infinite;
          animation: running-progress 2s cubic-bezier(0.4, 0, 0.2, 1) infinite;
        }
        @-webkit-keyframes running-progress {
          0% { margin-left: 0px; margin-right: 100%; }
          50% { margin-left: 25%; margin-right: 0%; }
          100% { margin-left: 100%; margin-right: 0; }
        }
        @keyframes running-progress {
          0% { margin-left: 0px; margin-right: 100%; }
          50% { margin-left: 25%; margin-right: 0%; }
          100% { margin-left: 100%; margin-right: 0; }
        }
    </style>
    <noscript>
        <style type="text/css">
            .pagecontainer {display:none;}
        </style>
        <div class="noscriptmsg">
        You don't have javascript enabled!
        Turn on javascript to continue to this page.
        </div>
    </noscript>
</head>
<body>
    <div class="main">
        <h2 style="color: white;">Creating DeepFake</h2>
        <p style="color: white;">Progress:</p>
        <div class="progress-line"></div>
    <div>
    
    <script>
      document.location.href = '/convertDf'
    </script>
</body>
</html>
  ''')
f.close()

In [ ]:
from flask import *
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from werkzeug.utils import secure_filename
import time, requests

app = Flask(__name__, root_path='/content/')
CORS(app)
run_with_ngrok(app)

@app.before_request
def before_request():
  global url
  url = request.url
  url = url.split('.ngrok.io')[0]
  url += '.ngrok.io'

@app.route('/')
def ip_get():
  if request.environ.get('HTTP_X_FORWARDED_FOR') is None:
    jsonIp = {'ip': request.environ['REMOTE_ADDR']}
  else:
    jsonIp = {'ip': request.environ['HTTP_X_FORWARDED_FOR']}

  print(jsonIp)
  return redirect('/main')

@app.route('/main')
def main():
  return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
  f = request.files['file']
  f.save(secure_filename(f.filename))
  ret = redirect(url + '/progress')
  return redirect(url + '/progress')

@app.route('/progress', methods=['POST', 'GET'])
def progress():
  return render_template('progress.html')

@app.route('/convertDf', methods=['POST', 'GET'])
def deepFake():
  time.sleep(10)
  return redirect('/main')

app.run()